In [1]:
# pip install kaggle if you're in a non-Kaggle env
# pip install simpletransformers if you have internet

In [2]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv

worker,less_toxic,more_toxic
313," This article sucks 

woo woo wooooooo","WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=
WHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back"


In [3]:
import pandas as pd
trainer = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")

toxic = trainer[["more_toxic"]]
toxic["text"] = toxic["more_toxic"]
toxic["label"] = 0.75

nontoxic = trainer[["less_toxic"]]
nontoxic["text"] = nontoxic["less_toxic"]
nontoxic["label"] = 0.35

combined = toxic.append(nontoxic).sample(frac=1, random_state=101)
combined = combined[["text", "label"]]
combined["label"] = combined["label"].astype('float')
combined.head(2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,text,label
27323,Wha's a reliable source mike? Something that c...,0.35
10783,Your contempt is noted. I hope I don't dissapp...,0.35


In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(combined, random_state=101, train_size=0.99)

In [5]:
# https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-2-word-vectors
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [6]:
def review_to_wordlist( txt, remove_stopwords=False ):
    txt = BeautifulSoup(txt).get_text()
    txt = re.sub("[^a-zA-Z]"," ", txt)
    words = txt.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)
review_to_wordlist(test['text'].to_list()[1])

['legend',
 'i',
 'saw',
 'one',
 'of',
 'his',
 'paintings',
 'at',
 'moma',
 'in',
 'ny',
 'and',
 'he',
 'actually',
 'put',
 'nuts',
 'bolts',
 'cig',
 'butts',
 'into',
 'the',
 'paint',
 'when',
 'he',
 'mixed',
 'his',
 'paint',
 'for',
 'his',
 'paintings',
 'pretty',
 'frickin',
 'amazing']

In [7]:
train_content = list(map(lambda sentence: review_to_wordlist(sentence), train['text'].to_list()))
train_content[0]

['i',
 'hope',
 'that',
 'one',
 'day',
 'one',
 'of',
 'your',
 'pupils',
 'drinks',
 'sulphuric',
 'acid',
 'that',
 'you',
 'have',
 'carelessly',
 'left',
 'out',
 'in',
 'a',
 'bottle',
 'of',
 'water',
 'it',
 'will',
 'be',
 'a',
 'shame',
 'but',
 'you',
 'will',
 'learn',
 'from',
 'it',
 'if',
 'it',
 'did',
 'happen',
 'hahahahahahahahahaaahahhaha']

In [8]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

from gensim.models import word2vec
model = word2vec.Word2Vec(train_content, workers=num_workers, \
            min_count = min_word_count, \
            window = context, sample = downsampling)

In [9]:
model.init_sims(replace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  """Entry point for launching an IPython kernel.


In [10]:
'hello' in model.wv

True

In [11]:
import numpy as np
indexes = []
def makeFeatureVec(words, num_features=100):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    for word in words:
        if word in model.wv: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model.wv[word])
    #if nwords < 1:
    #    print(words)
    vectors = np.divide(featureVec, nwords)
    if True in np.isnan(vectors):
        return np.zeros((num_features,),dtype="float32")
    return vectors
makeFeatureVec(['hello', 'love'])

array([-0.08887502, -0.02043539,  0.05550691,  0.01657881, -0.01528576,
       -0.1250256 , -0.02412267,  0.13341433, -0.01970973,  0.07730454,
       -0.01966708,  0.00626751,  0.12408589, -0.05059095, -0.01251615,
       -0.0143823 ,  0.06314801, -0.04658615, -0.0717314 , -0.13275701,
        0.00563906, -0.13381451,  0.06382346, -0.08975514, -0.07250064,
        0.07830334,  0.04491878, -0.06034213, -0.0290865 ,  0.07868766,
        0.0502138 ,  0.05728404, -0.05126774,  0.10934655, -0.03363477,
        0.01295422, -0.09753324, -0.10024247, -0.13740116, -0.02370553,
       -0.05579504,  0.14024974,  0.04341048,  0.0478991 , -0.02921975,
       -0.07167941, -0.07330202, -0.01164642, -0.0259811 ,  0.1737422 ,
       -0.05249663, -0.03726661,  0.05941698,  0.00793718, -0.07585806,
       -0.03135695, -0.12221043,  0.06239619, -0.04492736,  0.03230243,
       -0.0406142 , -0.01696925,  0.05929828,  0.01683241, -0.02506147,
       -0.03974362, -0.10198577,  0.04547145, -0.02871238, -0.02

In [12]:
trainDataVecs = list(map(lambda sentence: makeFeatureVec(sentence), train_content))
trainDataVecs[0]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


array([-3.8488768e-03,  4.6357837e-02, -3.5712216e-02,  6.5862560e-03,
       -1.7198015e-02,  8.8871811e-03, -4.8990253e-02, -5.9407886e-02,
       -4.7607772e-02, -3.2411363e-02,  2.7966309e-02,  1.7181924e-02,
       -2.5765558e-03, -2.4893992e-03, -1.8339515e-02,  1.7202878e-02,
        5.1218424e-02,  1.2517586e-02,  4.6584471e-03, -6.7705847e-03,
       -2.7721385e-02, -5.4791570e-02, -4.5337699e-02, -4.1793041e-02,
        1.3950695e-02, -3.2013729e-02, -2.8673930e-02,  1.8623186e-02,
        1.1020536e-02,  2.9372356e-03, -3.3720660e-03,  3.5466135e-02,
       -4.9687929e-02,  7.1218774e-02,  2.7552865e-02,  1.1592381e-02,
        3.2061145e-02,  5.1007695e-02,  2.3456274e-02,  4.3377034e-02,
       -1.0447245e-02,  2.4811639e-02, -1.2090638e-03,  1.0774211e-02,
       -2.6864983e-02,  3.2453786e-03,  3.3250399e-02, -2.9181666e-04,
       -2.9359998e-02,  1.6578805e-02,  1.3431702e-02,  8.2506724e-03,
       -1.0768864e-02,  2.3667051e-02,  4.0822580e-02, -6.0138751e-02,
      

In [13]:
test_content = list(map(lambda sentence: review_to_wordlist(sentence), test['text'].to_list()))
testDataVecs = list(map(lambda sentence: makeFeatureVec(sentence), test_content))

In [14]:
#from sklearn.ensemble import RandomForestRegressor
#forest = RandomForestRegressor( n_estimators = 100 )

from sklearn.ensemble import GradientBoostingRegressor
forest = GradientBoostingRegressor()
forest = forest.fit( trainDataVecs, train["label"] )

In [15]:
predictions = forest.predict( testDataVecs )

In [16]:
from sklearn.metrics import explained_variance_score
explained_variance_score(test['label'], predictions)

0.09773336216381323

In [17]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv

comment_id,text
114890,"""
 

Gjalexei, you asked about whether there is an """"anti-editorializing"""" policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.


In [18]:
import csv

texts = []
ids = []
with open('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv', 'r') as rfile:
  rdr = csv.reader(rfile)
  batch = None
  for row in rdr:
    if batch is None:
        batch = True
        continue
    ids.append(row[0])
    texts.append(row[1].replace('"', '').strip())

In [19]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv

comment_id,score
114890,0.5
732895,0.5
1139051,0.5
1434512,0.5


In [20]:
texts[0]

"Gjalexei, you asked about whether there is an anti-editorializing policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.\n\nIf something you write is edited and you're not sure why, please continue to question such edits on the talk page.  Sometimes, you'll learn more about wikipedia policy.  Sometimes, you'll find out that some other people working on here can get it flat-out wrong ) Robert Merkel"

In [21]:
predict_txts = list(map(lambda sentence: review_to_wordlist(sentence), texts))
predictDataVecs = list(map(lambda sentence: makeFeatureVec(sentence), predict_txts))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [22]:
op = []
id_idx = 0

with open('submission.csv', 'w') as opfile:
  writer = csv.writer(opfile)
  writer.writerow(["comment_id","score"])

  labels = forest.predict( predictDataVecs )
  for label in labels:
      writer.writerow([
        ids[id_idx],
        float(label)
      ])
      id_idx += 1

In [23]:
! head -n 5 submission.csv